In [1]:
#Retrieve and save data locally and if possible, upload to AWS S3.

This is a comprehensive overview of how to pre-treat medical lung images prior to them being fed into D.L. machine.

This is the outline if what will be accomplished:

1. **Loading the DICOM files** and adding missing metadata.
2. **Converting the pixel values to Hounsfield Units (HU),** and to what tissue these unit values correspond.
3. **Resampling** to an isomorphic resolution to remove variance in scanner resolution.
4. **3D plotting.** (Visualization is very useful to see what we are doing).
5. **Lung segmentation.**
6. **Normalization** that makes sense.
7. **Zero centering** the scans.

Importing pkgs and determining Pat's.

In [6]:
%matplotlib inline

import numpy as np
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

#Some constants
# INPUT_FOLDER = '../input/sample_images/'
# patients = os.listdir(INPUT_FOLDER)
# patients.sort()

## 1. Loading the DICOM files

Dicom 
- de-facto file standard for medical imaging. 
- contain a lot of metadata (pixel size, how long one pixel is in every dimension in the real world)
- pixel size/coarseness of scan differs from scan to scan (perform isomorphic resampling)

Following is code to load scan, which has multiple slices. 
This will be saved in a Python list.
Every folder in dataset is one scan (one patient).
One metadata field is missing, pixel size in Z direction, which is the slice thickness.
This will be inferred and added to metadata. 

In [10]:
#Load scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0]).SliceLocation - slices[1].SliceLocation
    
    for s in slices:
        s.SliceThickness = slice_thickness
                                        
    return slices

## 2. Converting the pixel values to Hounsfield Units (HU)

The unit of measurement in CT scans is **Hounsfield Unit (HU)** which is the measurement of radiodensity. CT scanners are carefully calibrated to accurately measure this. 

By default though, the returned values are not in this unit. This can be fixed.

Some scanners have cylindrical scanning bounds, but the output image is square. The pixels that fall outside of these bounds get the fixed value -2000. This will be set to 0, which corresponds to air. 

Getting back to HU units, we will multiply with rescale slope and adding the intercept (which is stored in metadata of the scans).

In [18]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    #Convert to int16 (from sometimes int16),
    #should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    
    #Set outside-of-scan pixels to 0
    #The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    #Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

Taking a look at one of the patients:

In [20]:
first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins = 80, color = 'c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# #Show some slice in the middle
plt.imshow(first_patient_pixels[80], cmp=plt.cm.gray)
plt.show()

In [23]:
first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
plt.hist(first_patient_pixels.flatten(), bins = 80, color = 'c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# #Show slice in the middle
plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
plt.show()

Based on Hounsfield Units and this histogram, clearly see which pixels are air and which are tissue. This will be used for lung segmentation.

## 3. Resampling

This is so that the full dataset has the same isotropic resolution between images.  

If choosing to resample everything to 1mm x 1mm x 1mm, can use 3d ConvNets without having concern about learning zoom/slice thickness 
variability. 

Though simple in plan, harder to code since there are some edge cases. However, following code flattens all images to the same dimensions.

In [25]:
def resample(image, scan, new_spacing = [1,1,1]):
    
    #Determine the current pixel spacing
    
    spacing = np.array([scan[0].SliceThickeness] + scan[0].PixelSpacing, dtype=np.float32)
    
    resize_factor = spacing/new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

When using this, remember to save the new spacing!
(The script picks the best spacing with rounding)

Now, let's resample patient's pixels to dimensions: 1mm x 1mm x 1mm.

In [28]:
pix_resampled, spacing = resample(first_patient_pixels, first_patient, [1,1,1])
print("Shape before resampling\t", first_patient_pixels.shape)
print("Shape after resampling\t", pix_resampled.shape)

## 4. 3D Plotting

For visualization, it is useful to show a 3D image of the scan. 
Using marching cubes to create an approximate mesh for 3D object,
and plot this with matplotlib.
This is not the most efficient, but it is the best for now.

In [29]:
def plot_3d(image, threshold=-300):
    
    #Position the scan upright,
    #so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.mearching_cubes(p, threshold)
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111,projection='3d')
    
    #Fancy indexing: 'verts[faces]' to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])
    
    plt.show()

Our plot function has a threshold argument that we can use to plot certain structures, such as all the tissue or in the case we have below, just the bones. In order to determine which structure to plot, refer to the Hounsfield units table mentioned above. 

In [31]:
# plot_3d(pix_resampled, 400)

## 5. Lung segmentazation

In order to reduce the noise and focus on the tissue we want to analyze/visualize, we can segment the lungs (and tissue around it). 

The method used consists of a series of applications of region growing and morphological operations. In this case, we use connected component analysis. 

The steps are:

* Threshold the image (-320 HU is a good threshold, but it does not matter much for this approach).
* Perform connected components, determine label of air around person, fill this with 1s in the binary image.
* Option: For every axial slice in the scan, determine the largest solid connected component (the body + air around the person), set others equal to 0. This fills the structures in the lungs in the mask.
* Only keep the largest air pocket (the human body has other air pockets here and there).

In [ ]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
    
def segment_lung_mask(image, fill_lung_structures=True):
    
    #not actually binary, but 1 and 2.
    #0 is treated as background, which we do not want
    
    binary_image = np.array(image > -320, dtype=mp.int8) + 1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    # Improvement: Pick multiple background labels from around the patient
    # More resistant to "trays" on which the patient lays cutting the air
    # around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    #Method of filling the lung structures (that is better than morphological closing)
    if fill_lung_structures:
        #For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg = 0)
            
            if l_ma is not None: #This slice contains some lung 
                binary_image[i][labeling != l_max] = 1
                
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image #Invert it, lungs are now 1
    
    #Remove other air pockets inside the body
    labels = measure.label(binary_image, background = 0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: #There are air pockets
        binary_image[labels != l_max] = 0
    
    
    return binary_image

In [ ]:
segmented_lungs = segment_lung_mask(pix_resampled, False)
segmented_lungs_fill = segment_lung_mask(pix_resampled, True)

In [ ]:
plot_3d(segmented_lungs, 0)

One thing to improve on here is to include structures within the lungs. We do not want just air in the lungs.

In [ ]:
plot_3d(segmented_lungs_fill, 0)

This is much better. We can also visualize the difference between the last two lungs shown.

In [2]:
plot_3d(segmented_lungs_fill - segmented_lungs, 0)

Pretty awesome!

When wanting to use this mask, remember to **first apply a dilation morphological operation** on it (i.e. with a circular kernel). This will expand the mask in all directions. The air + structures in the lung will not contain all the nodules. Those on the side, which is where the nodules appear most will be particularly missed. So expand the mask a little bit!

**This segmentation may fail for some edge cases.** This is because this method relies on the air outside of the patient not being connected with the air that is inside the lungs. If the patient has had a tracheostomy, this will not be the case. I am unsure if this data type is present in the dataset. Also, for noisy images such as those including a pacemaker, this method will not work. In this situation, the second largest air pocket in the body will be segmented. Check the fraction of the image that the mask corresponds to, which should be small in this case. One thing you can do in this situation is to first apply a morphological closing operation with a kernel a few mm in size to close these holes. An alternative to this would be to not use the mask at all to simplify things.

## Normalization

The values we have for substances in our images currently range from -1024 to around 2000. Anything above 400 is not relevant for our study since these ares simply bones with different radiodensity. A good treshold to normalize between is: -1000 and 400. The following codes for this.

In [3]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0

def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

## Zero Centering

The final step to this preprocessing is to center the data to zero so that the mean = 0. This is done by subtracting all pixels from the mean pixel value. 

To determine this mean, average all images in the whole dataset.

**Warning: Do not zero center with the mean per image, which has been done. CT scanners are calibrated to return acccurate HU measurements. Lower contrast or brightness for does not exist for these images though they exist for normal images, but this is not what is comprised of the dataset.

In [4]:
PIXEL_MEAN = 0.25 #pre-calculated from LUNA16 competition

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

NEXT: Images are now ready to be fed into CNN / other ML method. =)
Recommended to do these steps offline since it takes a while to process these images. It may take overnight. Remember to save your results! Lastly, right after loading, do normalization and zero centering in order to save storage space.